## Big Data Analytics Assignments 1 and 2

#### Problem 1: Simple DataFrame Queries

First mount the San Francisco fire data that we used for the DataBricks tutorial.  You can use the same file to mount the data that was previously used (in class).  You can verify that the data was uploaded properly by running the following command:

In [0]:
%fs ls /mnt/sf_open_data/fire_dept_calls_for_service/

path,name,size
dbfs:/mnt/sf_open_data/fire_dept_calls_for_service/Fire_Department_Calls_for_Service.csv,Fire_Department_Calls_for_Service.csv,1634673683


a. Read in the data and specify a proper schema.

In [0]:
# Your code here.

b. Fix the dates so that the dates are time-stamps.

In [0]:
# Your code here.

c. Verify the data is of the proper type.

In [0]:
# Your code here.

d. Answer this question by using data frames:  How many Units are there (as indicated by the number of unique UnitIDs). Hint: use count instead of show.

In [0]:
# Your code here.

e. Answer this question by using data frames: How many incidents were there at each zip code?

In [0]:
# Your code here.

f. Answer this question by using data frames: Show that all twelve months are accounted for in the Fire Service Calls data column.

In [0]:
# Your code here.

g. Answer this question by using data frames: How many service calls were placed each month in 2001?

In [0]:
# Your code here.

h. Answer this question by using data frames: Create a bar graph of your result from the previous part.

In [0]:
# Your code here.

i. Answer this question by using data frames: How many fires were ther in 2015 by month.  (Filter on CallTypeGroup == 'Fire' and add .show() to the end of your command)

In [0]:
# Your code here.

#### Problem 2: Simple SQL Queries

We can repartition the data so that the SQL queries go fast.  This part is done for you.

In [0]:
fireServiceCallsTsDF.rdd.getNumPartitions()
fireServiceCallsTsDF.repartition(6).createOrReplaceTempView("fireServiceVIEW");
spark.catalog.cacheTable("fireServiceVIEW")
spark.table("fireServiceVIEW").count()

Out[ 10 ]: 4091248

a. Why does the previous code chunk take a long time to run the first time you run it?

In [0]:
# Your code here.

b. Answer this question by using SQL:  How many Units are there (as indicated by the number of unique UnitIDs). Hint: use count instead of show.

In [0]:
# Your code here.

c. Answer this question by using SQL: How many incidents were there at each zip code?

In [0]:
# Your code here.

d. Answer this question by using SQL: Show that all twelve months are accounted for in the Fire Service Calls data column. Hint: use backticks (``) for variable names rather than quotes ('').

In [0]:
# Your code here.

e. Answer this question by using SQL: How many service calls were placed each month in 2001?

In [0]:
# Your code here.

f. Answer this question by using SQL: How many fires were there in 2015 by month. (Filter on CallTypeGroup == 'Fire' and use single quotes to identify text strings)

In [0]:
# Your code here.

#### Problem 3: Machine Learning in PySpark

In this problem, we will modify the dataset above so as to predict false alarms. We will need to do some formatting to get the data in the proper format.  In particular, we will need to make all variables numberic.

In [0]:
incidentsDF = spark.read.csv('/mnt/sf_open_data/fire_incidents/Fire_Incidents.csv', header=True, inferSchema=True).withColumnRenamed('Incident Number', 'IncidentNumber').withColumnRenamed('Primary Situation', 'PrimarySituation').cache()
incidentsDF.repartition(3).createOrReplaceTempView("incidentsVIEW");
spark.catalog.cacheTable("incidentsVIEW")

Note that it may be helpful to look at the tables at your disposal.  You can do this with the following command:

In [0]:
print(spark.catalog.listTables())

[Table(name=u'dfview', database=None, description=None, tableType=u'TEMPORARY', isTemporary=True), Table(name=u'fireserviceview', database=None, description=None, tableType=u'TEMPORARY', isTemporary=True), Table(name=u'incidentsview', database=None, description=None, tableType=u'TEMPORARY', isTemporary=True)]

a. Join the fire data set with the incidents data on IncidentNumber so that we only include data that has an incident number in both tables.  Name the resulting table "DF".

In [0]:
# Your code here.

b. How many rows does your query from part a contain?

In [0]:
# Your code here.

c. Create a new column in DF called "FA" which is a logical cast as an integer.  The number should be: DF.PrimarySituation == '700 false alarm or false call, other'.

In [0]:
# Your code here.

d. Verify that your new column has both 0's and 1's in it.

In [0]:
# Your code here.

e. Create a new data frame that consists of only the following variables: `FA`, `NumberofAlarms`, `Exposure Number`, `Fire Fatalities`, `Fire Injuries`.

In [0]:
# This command will make a PySpark SQL data frame which will allow you to use SQL
DF.createOrReplaceTempView("DFview")
model_data=spark.sql("Your code here.")

You can run the following code to create the machine learning pipeline: pipe.

In [0]:
from pyspark.ml.feature import VectorAssembler

vec_assembler = VectorAssembler(inputCols=["NumberofAlarms", "Exposure Number", "Fire Fatalities", "Fire Injuries"], outputCol="features")

from pyspark.ml import Pipeline

# Make the pipeline
pipe = Pipeline(stages=[vec_assembler])

f. You can plug in your model data you created from part e into the fit and transform functions.

In [0]:
piped_data = pipe.fit().transform()

Here we define our training and test sets.

In [0]:
# Split the data into training and test sets
training, test = piped_data.randomSplit([.6, .4])

Now we create our logistic regression object and fit our model.

In [0]:
# Import LogisticRegression
from pyspark.ml.classification import LogisticRegression

# Create a LogisticRegression Estimator
lr = LogisticRegression(labelCol='FA')

# Call lr.fit()
best_lr = lr.fit(training)

g. Enter in the name of the dataset inside the parentheses.  This data set is the one we are making predictions on.

In [0]:
# Use the model to predict the test set
test_results = best_lr.transform()

DataFrame[FA: int, NumberofAlarms: int, Exposure Number: int, Fire Fatalities: int, Fire Injuries: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

h. Show the first 10 rows of your test_results.

In [0]:
# Your code here.

i. Calulate the classification accuracy of the classifier.

In [0]:
# Your code here.